In [64]:
# ライブラリのインポート
import numpy as np
from sklearn.metrics import r2_score
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout, RepeatVector, TimeDistributed
import keras
import joblib

In [68]:
# どれくらいの期間を基に予測するか　（例：★ここでは24
window_size = 24
npyname = "kddi_06_new"
datedir = "230316"

In [69]:
# 交通量データの読込み
timeseries_aug = np.load("./"+datedir+'/'+ npyname+"_01param.npy", allow_pickle=True)

In [70]:
model = keras.models.load_model(datedir+'/'+npyname+'_model.h5')

In [71]:
# 読み込んだ交通量データの確認
timeseries_aug.shape

(720, 16814)

In [72]:
# 読み込んだ交通量データの確認
timeseries_aug

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [73]:
# データの確認
timeseries_aug[3][1]

0.0

In [74]:
# データの正規化
# データを0～1の範囲に正規化）
scaler = MinMaxScaler(feature_range=(0, 1))
train_data = scaler.fit_transform(timeseries_aug)
train_data

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [75]:
# データサイズの確認
train_data.shape

(720, 16814)

In [76]:
# データ構造を再確認 
print(window_size)

24


In [77]:
# （予測用）
x_test = []
# テスト用予測データ
for i in range(window_size, len(train_data), window_size):
    x_test.append(train_data[i-window_size:i, :])
x_test.append(train_data[i-window_size:i, :])
    

In [78]:
# データの確認（予測用）
x_test[0][3][1]

0.0

In [79]:
# numpy_arrayに変換
x_test = np.array(x_test)
print(x_test.shape)

(30, 24, 16814)


In [80]:
# 予測実行
predictions = model.predict(x_test)
#predictions = scaler.inverse_transform(predictions) 

1/1 [==============================] - 0s 329ms/step


In [81]:
# 予測結果を確認
print(predictions.shape)
predictions[0]

(30, 24, 16814)


array([[ 7.17828516e-06, -6.73769508e-04,  2.26796861e-03, ...,
        -2.78287660e-03,  1.28856231e-03,  5.81815280e-03],
       [ 7.03112455e-04, -3.30961542e-03,  9.14974371e-04, ...,
        -2.61206063e-03,  2.03201594e-03,  1.30928438e-02],
       [ 1.27784570e-03, -4.75327298e-03, -1.61825097e-03, ...,
        -2.15111068e-04,  4.20908397e-03,  1.75839663e-02],
       ...,
       [ 1.87778019e-03, -3.07758572e-04, -2.06965618e-02, ...,
         1.93620473e-03, -2.27195956e-03, -2.91968626e-03],
       [ 2.10442115e-04,  1.39489910e-03, -2.07841825e-02, ...,
         3.25118843e-03, -7.71508436e-04, -2.59569311e-03],
       [-4.56204871e-05, -1.03290426e-04, -1.03185540e-02, ...,
        -1.93590764e-04,  1.26739545e-03,  3.15290736e-03]], dtype=float32)

In [82]:
# 保存したScalerの読込み
scalers = joblib.load(datedir+'/'+npyname+'_scaler.gz')

In [83]:
predictions_traffic_volume = []
for cnt in range(predictions.shape[0]):
    # 正規化されたデータを逆変換
    tst_predictions = scalers.inverse_transform(predictions[cnt])
    predictions_traffic_volume.append(tst_predictions)

In [84]:
len(predictions_traffic_volume)

30

In [85]:
#予測結果の保存
np.save(datedir+'/'+ npyname + '_03res_param.npy', predictions_traffic_volume)

In [86]:
predictions_traffic_volume = []
for cnt in range(predictions.shape[0]):
    # 正規化されたデータを逆変換
    tst_predictions = predictions[cnt]
    predictions_traffic_volume.append(tst_predictions)

scaler = MinMaxScaler(feature_range=(0, 1))
np.save(datedir+'/'+ npyname + '_03res_weight.npy', predictions_traffic_volume)

In [87]:
#ConvNpyLoadPostgres ->